In [1]:
# h2o.shutdown()

In [2]:
import h2o
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/jupyter/.*')
h2o.init(nthreads = -1)


/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

H2O cluster uptime:,1 hours 6 minutes 58 seconds 662 milliseconds
H2O cluster version:,3.8.2.9
H2O cluster name:,H2O_started_from_R_laurend_fzg786
H2O cluster total nodes:,1
H2O cluster total free memory:,3.53 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,2
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


# Summary Notes

* use gbm (consider all options, for comparison)
* AUC to set threshold
* Compare machine learning models to regular group by models
* remove the customerID since that is the same as a row index
* todo: fill in missing values for TotalCharges column
* todo: compare different stratified functions and parameter to see if they stratify in the same way *(i.e. split_frame(ratios=None, destination_frames=None, seed=None), stratified_kfold_column(n_folds=3, seed=-1), stratified_split(test_frac=0.2, seed=-1, and fold_assignment="Stratified" for the GBM estimator)*

### questions
* should there be a learn_rate_annealing argument for python Grid Search?

In [3]:
# A small clean telecommunications sample dataset (https://www.ibm.com/communities/analytics/watson-analytics-blog/predictive-insights-in-the-telco-customer-churn-data-set/
telco_dataset = h2o.import_file("/Users/laurend/Desktop/projects/WA_Fn-UseC_-Telco-Customer-Churn.csv")


Parse Progress: [##################################################] 100%


In [4]:
# get a summary of the dataset
print telco_dataset.nacnt()
telco_dataset.describe()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0]
Rows:7,043 Cols:21

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
CBS,Bits,222,33.035713,21.2 KB,7.7945857
CX0,Zero Sparse Bits,2,0.2976190,254 B,0.0910306
C1N,1-Byte Integers (w/o NAs),352,52.380955,99.0 KB,36.343796
C2S,2-Byte Fractions,32,4.7619047,16.4 KB,6.0116043
C4S,4-Byte Fractions,32,4.7619047,30.1 KB,11.059861
CStr,String,32,4.7619047,105.5 KB,38.699123



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
127.0.0.1:54321,272.5 KB,7043.0,32.0,672.0
mean,272.5 KB,7043.0,32.0,672.0
min,272.5 KB,7043.0,32.0,672.0
max,272.5 KB,7043.0,32.0,672.0
stddev,0 B,0.0,0.0,0.0
total,272.5 KB,7043.0,32.0,672.0


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
type,string,enum,int,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,enum
mins,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.25,18.8,0.0
mean,NaN,0.504756495811,0.162146812438,0.483032798523,0.299588243646,32.3711486582,0.903166264376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592219224762,NaN,64.7616924606,2283.30044084,0.265369870794
maxs,NaN,1.0,1.0,1.0,1.0,72.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,118.75,8684.8,1.0
sigma,NaN,0.500012873647,0.36861160561,0.49974751072,0.45811016751,24.5594810231,0.295752231784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.491456924049,NaN,30.0900470977,2266.77136188,0.441561305122
zeros,0,3488,5901,3641,4933,11,682,3390,2421,3498,3088,3095,3473,2810,2785,3875,2872,1544,0,0,5174
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
0,7590-VHVEG,Female,0.0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0.0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [5]:
# what does each column tell you, make sure type is correct, and see if new features need to be created with strings
telco_dataset.columns

[u'customerID',
 u'gender',
 u'SeniorCitizen',
 u'Partner',
 u'Dependents',
 u'tenure',
 u'PhoneService',
 u'MultipleLines',
 u'InternetService',
 u'OnlineSecurity',
 u'OnlineBackup',
 u'DeviceProtection',
 u'TechSupport',
 u'StreamingTV',
 u'StreamingMovies',
 u'Contract',
 u'PaperlessBilling',
 u'PaymentMethod',
 u'MonthlyCharges',
 u'TotalCharges',
 u'Churn']

In [6]:
# check whether the customerID column is unique per row, if so use as an index and remove from the predictors
telco_dataset['customerID'].asfactor().unique().nrow


7043

In [7]:
# select all columns as predictors except the customerID (which is like an index) and the response column
features_list = list(telco_dataset.columns[1:-1])
response_name = 'Churn'

In [8]:
# specify the response column
response_col = telco_dataset['Churn']
# get a list of the categorical levels in your response column
print 'the response classes are:',response_col.levels()
print 'number of classes:', response_col.nlevels()
print ''
# check that the response column is already interpreted as a factor (i.e. enum/categorical)
print 'Is the response column a categorical:',response_col.isfactor()
# check that there are two levels in our response column:
response_col.nlevels()
print ''
# check for missing values in the training set and specifically the response column
print 'there are {0} missing values in the dataset'.format(telco_dataset.isna().sum())
print 'there are {0} missing values in the labels'.format(telco_dataset[response_col].isna().sum())
print ''

print 'check for class imbalace'
print '------------------------'
telco_dataset['Churn'].table()


the response classes are: [['No', 'Yes']]
number of classes: [2]

Is the response column a categorical: [True]

there are 11.0 missing values in the dataset
there are 0.0 missing values in the labels

check for class imbalace
------------------------


Churn,Count
No,5174
Yes,1869


In [9]:
print 'Question: How imbalanced is this dataset? Ans: no the dataset is not imbalanced'
num_train_samples = telco_dataset.shape[0]  # Total number of training samples
telco_dataset['Churn'].table()['Count']/num_train_samples

Question: How imbalanced is this dataset? Ans: no the dataset is not imbalanced


Count
0.73463
0.26537


In [10]:
for column_name in telco_dataset.columns[1:-1]:
    print telco_dataset[str(column_name)].table()
    print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
    

gender      Count
--------  -------
Female       3488
Male         3555

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  SeniorCitizen    Count
---------------  -------
              0     5901
              1     1142

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Partner      Count
---------  -------
No            3641
Yes           3402

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Dependents      Count
------------  -------
No               4933
Yes              2110

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  tenure    Count
--------  -------
       0       11
       1      613
       2      238
       3      200
       4      176
       5      133
       6      110
       7      131
       8      123
       9      119

[73 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
PhoneService      Count
--------------  -------
No                  682
Yes                6361

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~

In [11]:
# split your datasets
train, valid, test = telco_dataset.split_frame(ratios=[0.6,0.2], seed=1234)

In [12]:
# check that the split went as expected
print train.shape
print valid.shape
print test.shape
print train.shape[0] + valid.shape[0] + test.shape[0]

(4248, 21)
(1415, 21)
(1380, 21)
7043


In [13]:
# run gbm estimator with the default parameters to establish a benchmark model (set seed for reproducibility)
# import GBM
from h2o.estimators.gbm import H2OGradientBoostingEstimator
default_model = H2OGradientBoostingEstimator(seed = 1234)

# model_1 = H2OGradientBoostingEstimator(distribution='bernoulli',
#                                     ntrees=100,
#                                     max_depth=4,
#                                     learn_rate=0.1,
#                                     seed = 1234)

In [14]:
# train the default model
default_model.train(x=features_list, y=response_name, training_frame=train)


gbm Model Build Progress: [##################################################] 100%


In [15]:
# get the AUC for the training set
print 'train auc:', default_model.auc() 

# get the AUC for the validation set
default_perf_on_valid = default_model.model_performance(valid)
print 'validation auc:', default_perf_on_valid.auc()

train auc: 0.908830586543
validation auc: 0.847424663727


In [16]:
# using parameters from https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.Rmd
# this model overfits less on the training set
model_0 = H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    max_depth=4,
                                    learn_rate=0.01,
                                    stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC",
                                    sample_rate = 0.8,
                                    col_sample_rate = 0.8,
                                    seed = 1234,
                                    score_tree_interval = 10)

In [17]:
model_0.train(x=features_list, 
              y=response_name, 
              training_frame=train,
              validation_frame = valid)


gbm Model Build Progress: [##################################################] 100%


In [18]:
# get the AUC for the training set
print 'train auc:', model_0.auc() 

# get the AUC for the validation set
model_0_perf_on_valid = model_0.model_performance(valid)
print 'validation auc:', model_0_perf_on_valid.auc()

train auc: 0.868352080745
validation auc: 0.848603394534


## Grid Search
First we want to know what value of max_depth to use because it has a big impact on the model training time and optimal values depend strongly on the dataset. We'll do a quick Cartesian grid search to get a rough idea of good candidate max_depth values. Each model in the grid search will use early stopping to tune the number of trees using the validation set AUC, as before.

In [19]:
# using grid to get the best max_depths
hyper_params = {'max_depth' : range(1,30,2)}
search_criteria = {'strategy': "Cartesian"}

In [20]:
# todo: check why learn_rate_annealing breaks
from h2o.grid.grid_search import H2OGridSearch
gs_1 = H2OGridSearch(H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.05,
                                    # learn_rate_annealing = 0.99, 
                                    sample_rate = 0.8,
                                    col_sample_rate = 0.8,
                                    seed = 1234,
                                    score_tree_interval = 10,              
                                    stopping_rounds = 5,
                                    stopping_metric = "AUC",
                                    stopping_tolerance = 1e-4),
                                    hyper_params = hyper_params,
                                    grid_id = 'grid_determines_max_depth',
                                    search_criteria = search_criteria)

In [21]:
# train grid search
gs_1.train(x=features_list, 
           y=response_name, 
           training_frame=train,
           validation_frame = valid)


gbm Grid Build Progress: [##################################################] 100%


In [22]:
# get the grid search results to see which max_depth performed the best
print(gs_1)

     max_depth                           model_ids              logloss
0            1   grid_determines_max_depth_model_0   0.4028237180623319
1            3   grid_determines_max_depth_model_1  0.40541131949118514
2            5   grid_determines_max_depth_model_2  0.41012982432581363
3            7   grid_determines_max_depth_model_3   0.4201635329296258
4            9   grid_determines_max_depth_model_4  0.44099414547781474
5           11   grid_determines_max_depth_model_5   0.4502050937706764
6           13   grid_determines_max_depth_model_6  0.45941858208804986
7           17   grid_determines_max_depth_model_8   0.4638066131954029
8           15   grid_determines_max_depth_model_7   0.4638805716581195
9           23  grid_determines_max_depth_model_11   0.4650776614640909
10          19   grid_determines_max_depth_model_9  0.46636096088940876
11          21  grid_determines_max_depth_model_10   0.4666626470506088
12          27  grid_determines_max_depth_model_13   0.466870465

In [23]:
# print out the auc for all models, sorted from best to worst
auc_table = gs_1.sort_by('auc(valid=True)',increasing=False)
print(auc_table)


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,Hyperparameters: [max_depth],auc(valid=True)
grid_determines_max_depth_model_0,[1],0.8507848
grid_determines_max_depth_model_1,[3],0.8483866
grid_determines_max_depth_model_2,[5],0.8442249
grid_determines_max_depth_model_3,[7],0.8389147
grid_determines_max_depth_model_4,[9],0.8295361
grid_determines_max_depth_model_5,[11],0.8254979
grid_determines_max_depth_model_11,[23],0.8240235
grid_determines_max_depth_model_8,[17],0.8230958
grid_determines_max_depth_model_13,[27],0.8227580
grid_determines_max_depth_model_14,[29],0.8226148


In [24]:
# find the range of the max_depth for the top ten models
new_auc_table = auc_table[1:10]
max_depths_to_use = new_auc_table['Hyperparameters: [max_depth]']
print max_depths_to_use

# get the max depths as a list
new_maxmin_list = []
for element in max_depths_to_use:
    new_maxmin_list.append(element[0])
new_max = max(new_maxmin_list)
new_min = min(new_maxmin_list)

([3], [5], [7], [9], [11], [23], [17], [27], [29])


Now that we know a good range for max_depth, we can tune all other parameters in more detail. Since we don't know what combinations of hyper-parameters will result in the best model, we'll use random hyper-parameter search to "let the machine get luckier than a best guess of any human".

In [25]:
# variable used in the dictionary:
log_val = math.log(train.nrow,2)-1

# create hyperameter and search criteria lists
hyper_params_2 = {'max_depth' : list(np.arange(new_min,new_max,1)),
                'sample_rate': list(np.arange(0.3,1,0.01)),
                'col_sample_rate' : list(np.arange(0.3,1,0.01)),
                'col_sample_rate_per_tree': list(np.arange(0.3,1,0.01)),
                'col_sample_rate_change_per_level': list(np.arange(0.9,1.1,0.01)),
                'min_rows': list(2**np.arange(0,log_val,1)),
                'nbins': list(2**np.arange(4,10,1)),
                'nbins_cats': list(2**np.arange(4,12,1)),
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_2 = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 36000,
                   'max_models' :50,
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   }

In [26]:
gs_2 = H2OGridSearch(H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.05,
                                    #learn_rate_annealing = 0.99,
                                    stopping_rounds = 5,
                                    stopping_tolerance = 1e-4,
                                    stopping_metric = "AUC", 
                                    score_tree_interval = 10),
                                    hyper_params = hyper_params_2,
                                    grid_id = 'grid_2',
                                    search_criteria = search_criteria_2)

In [27]:
# train the grid and print results
gs_2.train(x=features_list, 
           y=response_name, 
           training_frame=train,
           validation_frame = valid)
print(gs_2)


gbm Grid Build Progress: [##################################################] 100%
          col_sample_rate col_sample_rate_change_per_level  \
0                    0.33               1.0700000000000003   
1      0.6300000000000003                             0.93   
2     0.46000000000000013               0.9700000000000001   
3      0.8700000000000006               0.9600000000000001   
4      0.7800000000000005                             0.91   
5      0.4500000000000001               1.0300000000000002   
6     0.46000000000000013               0.9800000000000001   
7      0.6500000000000004               1.0100000000000002   
8                    0.34                             0.92   
9                    0.33               1.0500000000000003   
10     0.7700000000000005                             0.93   
11     0.5600000000000003               1.0500000000000003   
12    0.35000000000000003                             0.91   
13     0.9300000000000006                       

In [28]:
# print out the auc for all of the models
auc_table_2 = gs_2.sort_by('auc(valid=True)',increasing=False)
print(auc_table_2)



Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [nbins, col_sample_rate, min_split_improvement, col_sample_rate_per_tree, min_rows, col_sample_rate_change_per_level, nbins_cats, sample_rate, histogram_type, max_depth]",auc(valid=True)
grid_2_model_49,"[32, 0.33, 1e-08, 0.4300000000000001, 512.0, 1.0700000000000003, 1024, 0.5200000000000002, u'QuantilesGlobal', 4]",0.8520805
grid_2_model_11,"[32, 0.46000000000000013, 1e-06, 0.47000000000000014, 512.0, 0.9700000000000001, 512, 0.7500000000000004, u'QuantilesGlobal', 12]",0.8518229
grid_2_model_10,"[32, 0.7800000000000005, 0.0001, 0.6400000000000003, 1024.0, 0.91, 32, 0.5700000000000003, u'QuantilesGlobal', 19]",0.8517322
grid_2_model_23,"[16, 0.8700000000000006, 0.0001, 0.4100000000000001, 1024.0, 0.9600000000000001, 32, 0.4500000000000001, u'UniformAdaptive', 4]",0.8516573
grid_2_model_34,"[64, 0.6500000000000004, 1e-08, 0.7800000000000005, 1024.0, 1.0100000000000002, 1024, 0.3, u'UniformAdaptive', 19]",0.8512789
---,---,---
grid_2_model_36,"[64, 0.37000000000000005, 1e-06, 0.8900000000000006, 2048.0, 0.9400000000000001, 128, 0.46000000000000013, u'QuantilesGlobal', 10]",0.8150076
grid_2_model_21,"[16, 0.46000000000000013, 1e-06, 0.6600000000000004, 2048.0, 1.04, 64, 0.9500000000000006, u'QuantilesGlobal', 4]",0.8147645
grid_2_model_0,"[256, 0.9000000000000006, 1e-08, 0.37000000000000005, 2048.0, 0.9500000000000001, 1024, 0.6300000000000003, u'RoundRobin', 28]",0.8144281
grid_2_model_19,"[16, 0.47000000000000014, 0.0, 0.9200000000000006, 2048.0, 1.0300000000000002, 1024, 0.8200000000000005, u'RoundRobin', 7]",0.8144176



See the whole table with table.as_data_frame()



In [29]:
# get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model('grid_2_model_41')
test_performance_model = best_model.model_performance(test)

# get the performance on the test model
print test_performance_model.auc()

0.859391718201


In [30]:
# save a csv of the predictions:
# first get the predictions from the best model
best_model_predictions = best_model.predict(test)
best_model_predictions.head()


gbm prediction Progress: [##################################################] 100%


predict,No,Yes
No,0.957239,0.0427606
No,0.961901,0.0380993
Yes,0.637631,0.362369
No,0.879035,0.120965
No,0.785426,0.214574
Yes,0.522067,0.477933
Yes,0.23311,0.76689
No,0.964092,0.0359077
Yes,0.465657,0.534343
No,0.78968,0.21032


In [32]:
# export predictions as a csv to the data analysis directory
# h2o.h2o.export_file(best_model_predictions, path ="https://s3.amazonaws.com/h2o-smalldata/TelcoChurn.csv")